In [61]:
import pandas as pd
import numpy as np
import pyreadr

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression

from scipy.stats import spearmanr
import xgboost as xgb

In [62]:
# specimens/subjects
specimens = pd.read_csv("subject_specimen.tsv", sep="\t", index_col=0)

# pbmcs
pbmc_freqs = pd.read_csv("pbmc_cell_frequency_batchCorrected_data.tsv", sep="\t", index_col=0).T
pbmc_tpms = pd.read_csv("pbmc_gene_expression_tpm_batchCorrected_data.tsv", sep="\t", index_col=0).T

# cytokines
cyto_olink = pd.read_csv("plasma_cytokine_concentrations_by_olink_batchCorrected_data.tsv", sep="\t", index_col=0).T
cyto_legend = pd.read_csv("plasma_cytokine_concentrations_by_legendplex_normalized_data.tsv", sep="\t", index_col=0).T

# plasma
plasma_ab = pd.read_csv("plasma_ab_titer_batchCorrected_data.tsv", sep="\t", index_col=0).T

# t cells
tcell_activ = pd.read_csv("t_cell_activation_raw_data.tsv", sep="\t").T
tcell_polar = pd.read_csv("t_cell_polarization_raw_data.tsv", sep="\t").T

def standardize_index(df):
    df.index = df.index.astype(int).astype(str)
    return df

specimens = standardize_index(specimens)
pbmc_freqs = standardize_index(pbmc_freqs)
pbmc_tpms = standardize_index(pbmc_tpms)
cyto_olink = standardize_index(cyto_olink)
cyto_legend = standardize_index(cyto_legend)
plasma_ab = standardize_index(plasma_ab)
tcell_activ = standardize_index(tcell_activ)
tcell_polar = standardize_index(tcell_polar)

cyto_olink = cyto_olink.add_suffix('_olink')
cyto_legend = cyto_legend.add_suffix('_legend')

# join
combined_df = specimens.copy()
other_dfs = [pbmc_freqs, pbmc_tpms, cyto_olink, cyto_legend, plasma_ab, tcell_activ, tcell_polar]

for df in other_dfs:
    combined_df = combined_df.join(df, how='left')

# problem 1.2 prep
day0_df = combined_df[combined_df['timepoint'] == 0].set_index('subject_id')
day14_df = combined_df[combined_df['timepoint'] == 14].set_index('subject_id')

merged_df = day0_df[['IgG_PT']].rename(columns={'IgG_PT': 'IgG_PT_day0'}).join(
    day14_df[['IgG_PT']].rename(columns={'IgG_PT': 'IgG_PT_day14'}),
    how='inner'
)

merged_df['fold_change_IgG_PT'] = (merged_df['IgG_PT_day14'] + 1e-6) / (merged_df['IgG_PT_day0'] + 1e-6)

combined_df = combined_df.merge(merged_df[['fold_change_IgG_PT']], left_on='subject_id', right_index=True, how='left')
combined_df['fold_change_IgG_PT'] = np.where(combined_df['timepoint'] == 14, combined_df['fold_change_IgG_PT'], np.nan)

# problem 2.2 prep
day0_df = combined_df[combined_df['timepoint'] == 0].set_index('subject_id')
day1_df = combined_df[combined_df['timepoint'] == 1].set_index('subject_id')

merged_df = day0_df[['Monocytes']].rename(columns={'Monocytes': 'Monocytes_day0'}).join(
    day1_df[['Monocytes']].rename(columns={'Monocytes': 'Monocytes_day1'}),
    how='inner'
)

merged_df['fold_change_Monocytes'] = (merged_df['Monocytes_day1'] + 1e-6) / (merged_df['Monocytes_day0'] + 1e-6)

combined_df = combined_df.merge(merged_df[['fold_change_Monocytes']], left_on='subject_id', right_index=True, how='left')
combined_df['fold_change_Monocytes'] = np.where(combined_df['timepoint'] == 1, combined_df['fold_change_Monocytes'], np.nan)

# problem 3.2 prep
day0_df = combined_df[combined_df['timepoint'] == 0].set_index('subject_id')
day3_df = combined_df[combined_df['timepoint'] == 1].set_index('subject_id')

merged_df = day0_df[['ENSG00000277632.1']].rename(columns={'ENSG00000277632.1': 'CCL3_day0'}).join(
    day3_df[['ENSG00000277632.1']].rename(columns={'ENSG00000277632.1': 'CCL3_day3'}),
    how='inner'
)

merged_df['fold_change_CCL3'] = (merged_df['CCL3_day3'] + 1e-6) / (merged_df['CCL3_day0'] + 1e-6)

combined_df = combined_df.merge(merged_df[['fold_change_CCL3']], left_on='subject_id', right_index=True, how='left')
combined_df['fold_change_CCL3'] = np.where(combined_df['timepoint'] == 3, combined_df['fold_change_CCL3'], np.nan)

# problem 4.1 prep
day0_df = combined_df[combined_df['timepoint'] == 0].set_index('subject_id')
day30_df = combined_df[combined_df['timepoint'] == 30].set_index('subject_id')

merged_df = day0_df[['PT_P01579', 'PT_P05113']].rename(columns={'PT_P01579': 'PT_P01579_day0', 'PT_P05113': 'PT_P05113_day0'}).join(
    day30_df[['PT_P01579', 'PT_P05113']].rename(columns={'PT_P01579': 'PT_P01579_day30', 'PT_P05113': 'PT_P05113_day30'}),
    how='inner'
)

merged_df['fold_change_PT_ratio'] = ((merged_df['PT_P01579_day30'] + 1e-6) / (merged_df['PT_P05113_day30'] + 1e-6)) / ((merged_df['PT_P01579_day0'] + 1e-6) / (merged_df['PT_P05113_day0'] + 1e-6))

combined_df = combined_df.merge(merged_df[['fold_change_PT_ratio']], left_on='subject_id', right_index=True, how='left')
combined_df['fold_change_PT_ratio'] = np.where(combined_df['timepoint'] == 30, combined_df['fold_change_PT_ratio'], np.nan)

print(combined_df.shape)
print(combined_df.isnull().sum().sum())
print(combined_df.isnull().sum().sort_values(ascending=False).head())

(896, 6751)
1566692
fold_change_PT_ratio     853
fold_change_Monocytes    822
fold_change_CCL3         803
fold_change_IgG_PT       785
PHA                      664
dtype: int64


## Training

In [73]:
def predict(combined_df, target_col, train_datasets=[0, 1], test_dataset=2, target_timepoint=1):
    df = combined_df.copy()
    
    # encode categorical variables
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    le = LabelEncoder()
    for col in categorical_cols:
        df[col] = le.fit_transform(df[col].astype(str))
    
    train_df = df[df['dataset'].isin(train_datasets)]
    
    # handle test dataset
    if test_dataset == 3:
        # for dataset 3, get subjects from their existing timepoints
        test_df = df[df['dataset'] == test_dataset]
        # track subject_ids in the same order as the data
        subject_ids = test_df['subject_id'].values
        X_test = test_df.drop(columns=[target_col, 'subject_id', 'dataset', 'timepoint'], errors='ignore')
        y_test = None
    else:
        test_df = df[(df['dataset'] == test_dataset) & (df['timepoint'] == target_timepoint)]
        test_df = test_df.dropna(subset=[target_col])
        X_test = test_df.drop(columns=[target_col, 'subject_id', 'dataset', 'timepoint'])
        y_test = test_df[target_col]
        subject_ids = None
    
    # remove rows with NaN in target column for training
    train_df = train_df.dropna(subset=[target_col])
    
    # prepare features and target for training
    X_train = train_df.drop(columns=[target_col, 'subject_id', 'dataset', 'timepoint'])
    y_train = train_df[target_col]
    
    # remove columns with all NaN values
    X_train = X_train.dropna(axis=1, how='all')
    
    # ensure X_train and X_test have the same columns
    common_columns = list(set(X_train.columns) & set(X_test.columns))
    X_train = X_train[common_columns]
    X_test = X_test[common_columns]
    
    # impute missing values
    imputer = SimpleImputer(strategy='median')
    X_train_imputed = pd.DataFrame(
        imputer.fit_transform(X_train),
        columns=X_train.columns,
        index=X_train.index
    )
    X_test_imputed = pd.DataFrame(
        imputer.transform(X_test),
        columns=X_test.columns,
        index=X_test.index
    )
    
    # feature selection
    selector = SelectKBest(f_regression, k=min(100, X_train_imputed.shape[1]))
    X_train_selected = selector.fit_transform(X_train_imputed, y_train)
    X_test_selected = selector.transform(X_test_imputed)
    
    selected_feature_names = X_train_imputed.columns[selector.get_support()].tolist()
    
    # train model
    model = xgb.XGBRegressor(
        max_depth=6,
        learning_rate=0.01,
        n_estimators=200,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=3,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1,
        objective='reg:squarederror',
        random_state=42
    )
    model.fit(X_train_selected, y_train)
    
    # predict
    y_pred = model.predict(X_test_selected)
    
    print(f"\nPredicting {target_col} for dataset {test_dataset}")
    
    if test_dataset == 3:
        prediction_df = pd.DataFrame({
            'Subject': subject_ids,
            'Predicted_Value': y_pred
        })
        prediction_df['Predicted_Rank'] = prediction_df['Predicted_Value'].rank(method='dense', ascending=False)
        
        # get unique predictions (one per subject)
        prediction_df = prediction_df.groupby('Subject')['Predicted_Value'].mean().reset_index()
        prediction_df['Predicted_Rank'] = prediction_df['Predicted_Value'].rank(method='dense', ascending=False)
        
        print("\nPredicted Values with Subject IDs and Ranks:")
        print(prediction_df)
        return prediction_df
    else:
        y_pred_ranks = pd.Series(y_pred).rank(method='dense', ascending=False)
        y_test_ranks = y_test.rank(method='dense', ascending=False)
        spearman_corr, p_value = spearmanr(y_pred_ranks, y_test_ranks)
        
        comparison_df = pd.DataFrame({
            'Actual_Value': y_test.values,
            'Predicted_Value': y_pred,
            'Actual_Rank': y_test_ranks.values,
            'Predicted_Rank': y_pred_ranks.values
        })
        print("\nPredicted vs Actual Rankings:")
        print(comparison_df)
        
        print(f"\nSpearman Rank Correlation: {spearman_corr}")
        print(f"P-value: {p_value}")
        
        importance = model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': selected_feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        print("\nTop 10 most important features:")
        print(feature_importance.head(10))
        
        return comparison_df

In [ ]:

# Problem 1
#predict(combined_df, "IgG_PT", train_datasets=[0, 1], test_dataset=2, target_timepoint=1)
#predict(combined_df, "fold_change_IgG_PT", train_datasets=[0, 1], test_dataset=2, target_timepoint=14)

# Problem 2
#predict(combined_df, "Monocytes", train_datasets=[0, 1], test_dataset=2, target_timepoint=1)
#predict(combined_df, "fold_change_Monocytes", train_datasets=[0, 1], test_dataset=2)

# Problem 3
#predict(combined_df, "ENSG00000277632.1", train_datasets=[0, 1], test_dataset=2, target_timepoint=3)
#predict(combined_df, "fold_change_CCL3", train_datasets=[0, 1], test_dataset=2, target_timepoint=3)

# Problem 4
#predict(combined_df, "fold_change_PT_ratio", train_datasets=[0, 1], test_dataset=2, target_timepoint=30)

In [89]:
# read template first to get structure and SubjectID order
template_df = pd.read_csv("3rdChallengeSubmissionTemplate_10032024.tsv", sep='\t')

# make predictions for each column in order
predictions = {
    '1.1) IgG-PT-D14-titer-Rank': predict(combined_df, "IgG_PT", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=14),
    '1.2) IgG-PT-D14-FC-Rank': predict(combined_df, "fold_change_IgG_PT", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=14),
    '2.1) Monocytes-D1-Rank': predict(combined_df, "Monocytes", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=1),
    '2.2) Monocytes-D1-FC-Rank': predict(combined_df, "fold_change_Monocytes", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=1),
    '3.1) CCL3-D3-Rank': predict(combined_df, "ENSG00000277632.1", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=3),
    '3.2) CCL3-D3-FC-Rank': predict(combined_df, "fold_change_CCL3", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=3),
    '4.1) IFNG/IL5-Polarization-D30-Rank': predict(combined_df, "fold_change_PT_ratio", train_datasets=[0, 1, 2], test_dataset=3, target_timepoint=30)
}

# fill in template with predictions, maintaining template's SubjectID order
for col in template_df.columns:
    if col in predictions:
        rank_dict = dict(zip(predictions[col]['Subject'], predictions[col]['Predicted_Rank']))
        template_df[col] = template_df['SubjectID'].map(rank_dict)

# save final submission
template_df.to_csv("final_submission.tsv", sep='\t', index=False)

print("Final submission dataframe:")
print(template_df)


Predicting IgG_PT for dataset 3

Predicted Values with Subject IDs and Ranks:
    Subject  Predicted_Value  Predicted_Rank
0       119         2.767159            18.0
1       120         3.370854            12.0
2       121         2.119967            29.0
3       122         1.151930            54.0
4       123         1.167759            53.0
5       124         2.179653            28.0
6       125         3.038510            17.0
7       126         2.083534            30.0
8       127         1.890515            35.0
9       128         2.624708            20.0
10      129         2.200876            26.0
11      130         1.591540            43.0
12      131         1.505734            48.0
13      132         3.319011            14.0
14      133         1.466566            50.0
15      134         1.552343            46.0
16      135         2.341607            24.0
17      136         2.187835            27.0
18      137         1.509873            47.0
19      138         1

/Users/lucas/miniforge3/envs/cse258/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:331: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)



Predicting Monocytes for dataset 3

Predicted Values with Subject IDs and Ranks:
    Subject  Predicted_Value  Predicted_Rank
0       119         0.848832            43.0
1       120         0.621653            52.0
2       121         0.790484            47.0
3       122         1.044806            23.0
4       123         1.261365            10.0
5       124         0.633215            51.0
6       125         0.583881            53.0
7       126         1.240154            11.0
8       127         1.134739            18.0
9       128         0.782357            48.0
10      129         0.815508            46.0
11      130         1.199021            13.0
12      131         1.124984            19.0
13      132         1.092880            20.0
14      133         1.363587             6.0
15      134         1.080786            21.0
16      135         1.481239             3.0
17      136         0.744190            49.0
18      137         0.853664            41.0
19      138       

/Users/lucas/miniforge3/envs/cse258/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:331: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)



Predicting fold_change_PT_ratio for dataset 3

Predicted Values with Subject IDs and Ranks:
    Subject  Predicted_Value  Predicted_Rank
0       119        23.035124            33.0
1       120         7.351058            46.0
2       121        28.406153            20.0
3       122        23.699484            30.0
4       123        24.586931            27.0
5       124        23.580454            31.0
6       125        29.103678            13.0
7       126         3.491981            51.0
8       127        31.501257             7.0
9       128         2.820674            53.0
10      129         7.644797            45.0
11      130        51.133350             1.0
12      131        27.666458            24.0
13      132         4.626792            50.0
14      133        24.472082            28.0
15      134        16.553694            37.0
16      135         9.942683            39.0
17      136        29.088099            14.0
18      137        28.836905            16.0
19     